In [ ]:
!git clone https://github.com/resemble-ai/Resemblyzer.git

Cloning into 'Resemblyzer'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 595 (delta 2), reused 1 (delta 0), pack-reused 586
Receiving objects: 100% (595/595), 101.46 MiB | 47.25 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
!pip install webrtcvad-wheels

     |████████████████████████████████| 92kB 3.8MB/s 


In [ ]:
cd Resemblyzer

/content/Resemblyzer


In [ ]:
!pip3 install spectralcluster
!pip install git+https://github.com/jiaaro/pydub.git@master
!pip install kneed

  Cloning https://github.com/jiaaro/pydub.git (to revision master) to /tmp/pip-req-build-dpvw443c
  Running command git clone -q https://github.com/jiaaro/pydub.git /tmp/pip-req-build-dpvw443c
  Created wheel for pydub: filename=pydub-0.24.1-py2.py3-none-any.whl size=30880 sha256=9ac41d6131a126f57063acc4e5a8fa4a32f997058eb7c8faeca7406a6df50690
  Stored in directory: /tmp/pip-ephem-wheel-cache-lv2mc8l_/wheels/21/42/ed/000068980b52206c1f0fe8ca178cc01a7e094e366bc2cfb2e8
Successfully built pydub


In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from spectralcluster import SpectralClusterer
from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import write
from kneed import KneeLocator
from sklearn.cluster import KMeans

#Input audio file
audio_file_path = '/content/drive/My Drive/Dual Rnn/test.wav'
wav_fpath = Path(audio_file_path)

#Pre-processing the input audio file
wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder()
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)

#Output denoised wave after removing the pauses
write('/content/drive/My Drive/Dual Rnn/test1.wav', 16000, wav)

#Finding optimum clusters c
inertia = []
K = range(1, 15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(cont_embeds)
    inertia.append(km.inertia_)
kn = KneeLocator(list(K), inertia, S=1.0, curve='convex', direction='decreasing')
c=kn.knee
#c=2


#Passing the clusters to get the labels
clusterer = SpectralClusterer(
    min_clusters=c,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)


#Function to create the labelllings
def create_labelling(labels,wav_splits):
    #from resemblyzer import sampling_rate
    #import resemblyzer
    sampling_rate=16000
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling

labelling = create_labelling(labels,wav_splits)

#Creating different audios based on the labelling/timeframes received.
output = {}
for x, y, z in labelling:
    if x in output:
        output[x].append((y, z))
    else:
        output[x] = [(y, z)]

#keys = []
values = []
items = output.items()
for item in items:
    values.append(item[1])

audio = AudioSegment.from_wav('/content/drive/My Drive/Dual Rnn/test1.wav')
voices = []
n = 0
for i in values:
    for j in i:
        start_time, stop_time = j
        n += audio[(start_time*1000):(stop_time*1000)]
    voices.append(n)

for i, j in enumerate(voices):
    j.export(f'/content/drive/My Drive/Dual Rnn/voice_{i}.wav')

Loaded the voice encoder model on cuda in 0.01 seconds.


In [ ]:
c

4